In [1]:
import pandas as pd
import numpy as np

In [223]:
# Simulate experiment
p = [0.2, 0.5, 0.45, 0.3, 0.1, 0.45]
n = 1000

def sim_experiment(p, n):

    # k = np.argmax(np.random.beta(a, b))
    #     counts[k] += 1

    #     y = np.random.binomial(1, p[k])
    #     #update the prior
    #     a[k] += y
    #     b[k] += (1 - y)
    
    #flat priors
    a = 1 
    b = 1
    counts = np.zeros(len(p))
    
    results = []
    beta = []
    counts_rate = []

    for i in range(0, 6):
        #get number of successes for arm i
        k = np.random.binomial(n, p[i])
        counts[i] += 1
        #update the prior
        a = 1 + k
        b = 1 + n - k
        #save parameters of beta distribution
        beta.append((a, b))
        #calculate posterior mean and sd
        post_mean = a/(a+b)
        post_sd = (a*b/((a+b)**2*(a+b+1)))**0.5
        counts_rate.append(counts)
        results.append((i, p[i], k, post_mean, post_sd))

    return results, beta, counts_rate[0]

#run the experiment
results, beta, counts_rate = sim_experiment(p, n)
results

[(0, 0.2, 196, 0.1966067864271457, 0.012549108239437615),
 (1, 0.5, 504, 0.5039920159680639, 0.0157872212483476),
 (2, 0.45, 414, 0.41417165668662675, 0.015553384588814712),
 (3, 0.3, 320, 0.3203592814371258, 0.014733566380863672),
 (4, 0.1, 103, 0.10379241516966067, 0.0096302297867534),
 (5, 0.45, 446, 0.44610778443113774, 0.015695749724776158)]

In [224]:
counts_rate

array([1., 1., 1., 1., 1., 1.])

B

In [151]:
#get post probabilities that arm i is best
samples = np.array([np.random.beta(beta[i][0],beta[i][1], size=100000) for i in range(0, 6)])
# Count how many times each arm has the max sample
best_counts = (samples == samples.max(axis=0)).sum(axis=1)
posterior_best_probs = best_counts / 100000
pd.DataFrame(posterior_best_probs)

,0
0,0.00000
1,0.99913
2,0.00077
3,0.00000
4,0.00000
5,0.00010


C

In [190]:
# Simulate experiment
def thompson_experiment(p, n):
    #flat priors
    a = np.ones(len(p))
    b = np.ones(len(p))
    counts = np.zeros(len(p))

    results_thompson = []
    betas_thompson = []
    counts_rate = []
    regret_thom = []

    for _ in range(6000):
        #get number of successes for arm i
        k = np.argmax(np.random.beta(a, b))
        counts[k] += 1

        y = np.random.binomial(1, p[k])
        #update the prior
        a[k] += y
        b[k] += (1 - y)
        #save parameters of beta distribution

    post_means = a / (a + b)
    post_sds  = np.sqrt(a * b / ((a + b)**2 * (a + b + 1)))
    results_thompson.append((post_means, post_sds))
    counts_rate.append(counts)

    betas_thompson.append((a, b))

    return results_thompson, betas_thompson[0], counts_rate[0]

p = [0.2, 0.5, 0.45, 0.3, 0.1, 0.45]
n = 1000

results_thompson, betas_thompson, counts_rate = thompson_experiment(p, n)
results_thompson

[(array([0.2       , 0.49507817, 0.46222222, 0.11111111, 0.11764706,
         0.38461538]),
  array([0.07184212, 0.00694545, 0.0191758 , 0.07209841, 0.07594085,
         0.05072157]))]

In [191]:
counts_rate

array([  28., 5179.,  673.,   16.,   15.,   89.])

In [154]:
#get post probabilities that arm i is best
samples = np.array([np.random.beta(betas_thompson[0][i],betas_thompson[1][i], size=100000) for i in range(0, 6)])
# Count how many times each arm has the max sample
best_counts = (samples == samples.max(axis=0)).sum(axis=1)
posterior_best_probs = best_counts / 100000

fractions = counts_rate / 6000
posterior_best_probs

array([1.620e-03, 9.540e-01, 2.420e-02, 6.990e-03, 4.200e-04, 1.277e-02])

In [155]:
fractions

array([0.00416667, 0.6945    , 0.18583333, 0.015     , 0.00516667,
       0.09533333])

D

In [231]:
iterations = []
results_iter = []
rates = []
top_mean = []
regret = []
for _ in range(100):
    results,betas, counts_rate = sim_experiment(p, n)
    # results_iter.append(np.argmax(results[0]))

    probs = np.sum(counts_rate*p/6)
    regret.append(max(p) - probs)

    #to get highest posterior probility
    #get post probabilities that arm i is best
    samples = np.array([np.random.beta(betas[i][0],betas[i][1], size=100) for i in range(0, 6)])
    
    # Count how many times each arm has the max sample
    best_counts = (samples == samples.max(axis=0)).sum(axis=1)

    iterations.append(np.argmax(best_counts) + 1)

    rate_iter = best_counts.max() / 100
    rates.append(rate_iter)

iterations_rand = np.array(iterations)
iterations_rand

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2,
       2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2])

In [232]:
np.mean(regret)

np.float64(0.16666666666666674)

In [157]:
#get dataframe of the fraction of times each arm appears in list
df = pd.DataFrame()
df['true_prob'] = p
# create column of the fraction of times each arm value is in list iterations
df['fraction'] = [list(iterations_rand).count(i)/100 for i in range(1,7)]
df


,true_prob,fraction
0,0.20,0.00
1,0.50,0.95
2,0.45,0.02
3,0.30,0.00
4,0.10,0.00
5,0.45,0.03


In [ ]:
# samples = np.random.beta(
#     betas_thompson[0][np.newaxis, :],
#     betas_thompson[1][np.newaxis, :],
#     size=(100, 6)
# )
# best_idx = np.argmax(samples, axis=1)   # length-M array of ints in 0..K-1

# # 3) count and normalize
# counts = np.bincount(best_idx, minlength=6)
# posterior_best_prob = counts / 100
# results_iter.append(best_idx)

In [198]:
iterations = []
results_iter = []
rates = []
regret = []
for _ in range(100):
    results_thompson,betas_thompson, counts_rate = thompson_experiment(p, n)
    
    probs = np.sum(counts_rate*p/6000)
    regret.append(max(p) - probs)

    #get post probabilities that arm i is best
    samples = np.array([np.random.beta(betas_thompson[0][i],betas_thompson[1][i], size=(100)) for i in range(6)])
    # reg = np.array(max([p[i] - np.mean(samples[i]) for i in range(6)]))
    # regret.append(reg)
    
    # Count how many times each arm has the max sample
    best_counts = (samples == samples.max(axis=0)).sum(axis=1)
    results_iter.append(best_counts)

    # probs = np.sum(best_counts*p/100)
    # regret.append(max(p) - probs)

    # iteration_rand2 = iterations_rand - 1
    # p_values = np.array([p[i] for i in iteration_rand2])
    # max(p)-np.mean(p_values)

    # iterations.append(best_counts.max())
    iterations.append(np.argmax(best_counts)+1)
    rate_iter = best_counts.max() / 100
    rates.append(rate_iter)

iterations_thom = np.array(iterations)
iterations_thom

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 6, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [199]:
np.mean(regret)

np.float64(0.013098416666666668)

In [169]:
best_counts*p

array([ 0.  , 45.  ,  1.35,  0.  ,  0.  ,  3.15])

In [161]:
probs = best_counts*p/100
np.mean(max(p) - probs[probs!=0])


np.float64(0.3348333333333333)

In [162]:
#get dataframe of the fraction of times each arm appears in list
df = pd.DataFrame()
df['true_prob'] = p
# create column of the fraction of times each arm value is in list iterations
df['fraction'] = [list(iterations_thom).count(i)/100 for i in range(1,7)]
df

,true_prob,fraction
0,0.20,0.0
1,0.50,1.0
2,0.45,0.0
3,0.30,0.0
4,0.10,0.0
5,0.45,0.0


E

In [163]:
iteration_rand2 = iterations_rand - 1
p_values = np.array([p[i] for i in iteration_rand2])
max(p)-np.mean(p_values)


np.float64(0.0025000000000000022)

In [164]:
iterations_thom2 = iterations_thom - 1
p_values = np.array([p[i] for i in iterations_thom2])
max(p)-np.mean(p_values)

np.float64(0.0)